### What this script does 

- Loads all 10-min merged sonic/mast/radiation datasets and 10-min rain files.

- Merges rain onto the sonic time series.

- Flags/filters out rainy or inconsistent records (temp/wind sanity checks).

- Computes u*, θ*, Monin–Obukhov length L, ζ = z/L, vertical gradients, and stability functions ϕm and ϕh.

- Compares measured ϕm, ϕh to Businger–Dyer curves (stable/unstable), reports deviations, and plots nice side-by-side panels.

- Saves the final figure to disk.

#### Lines you’ll likely need to edit:
1) Your data locations:

    - DATA_ROOT = r"C:\path\to\your\Sonic"   <-- change if Sonic path differs
    - RAIN_ROOT = r"C:\path\to\your\Cloud_radar"  <-- change if Rain path differs

2) Which months to include (folders like 2024-03, 2024-04, …)
   
   DATE_GLOB = '2024-0[3-5]'  <-- e.g., '2024-0[3-6]' to include June

3) Site / constants (use your sonic height)
- Z_SONIC = 2.99     <-- set to your sonic height (m)
- KAPPA, G, theta0 = 0.4, 9.81, 288  <--leave unless you need different constants

4) Expected file names inside rain folders: rain_pattern = os.path.join(RAIN_ROOT, DATE_GLOB, '**', 'Rain_10min_Averages.csv')<-- change 'Rain_10min_Averages.csv' if your filename differs

5) QC thresholds (tune if needed)

6) Optional clear-sky filter 

7) Output plot location/name: output_path = os.path.join(DATA_ROOT, 'stability_functions.jpg')  <-- change name/path if desired

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import Polynomial


In [ ]:
#Edit before running!!!
# 1) Your data locations
# Where your merged_data_10min.csv files live (tower/sonic)
# Example: DATA_ROOT = r"D:\MyThesis\data\Sonic"
DATA_ROOT = r"C:\path\to\your\Sonic"      # <-- change if Sonic path differs
# Where your rain files live
# Example: RAIN_ROOT = r"D:\MyThesis\data\Cloud_radar"
RAIN_ROOT = r"C:\path\to\your\Cloud_radar" # <-- change if Rain path differs


# Date range of interest (folders named 2024-03, 2024-04, 2024-05, 2024-06)
DATE_GLOB = '2024-0[3-5]'

# Sonic height, von Kármán constant, gravity
Z_SONIC = 2.99
KAPPA, G,theta0 = 0.4, 9.81, 288

# ── 1) LOAD ALL TOWER/SONIC DATA (10-min merged) ───────────────────────────────

sonic_pattern = os.path.join(DATA_ROOT, '**', 'merged_data_10min.csv')
sonic_files = glob.glob(sonic_pattern, recursive=True)
if not sonic_files:
    raise RuntimeError(f"No sonic files found under {DATA_ROOT}")

df = pd.concat(
    (pd.read_csv(fn, parse_dates=['TIMESTAMP']) for fn in sonic_files),
    ignore_index=True
).sort_values('TIMESTAMP')
print(f"Loaded {len(sonic_files)} sonic files → {len(df)} rows")

# ── 2) LOAD ALL RAIN DATA (10-min averages) ────────────────────────────────────

rain_pattern = os.path.join(RAIN_ROOT, DATE_GLOB, '**', 'Rain_10min_Averages.csv')
rain_files = glob.glob(rain_pattern, recursive=True)
if not rain_files:
    raise RuntimeError(f"No rain files found under {RAIN_ROOT}/{DATE_GLOB}")

rain = pd.concat(
    (pd.read_csv(fn, parse_dates=['TIMESTAMP']) for fn in rain_files),
    ignore_index=True
).sort_values('TIMESTAMP')
print(f"Loaded {len(rain_files)} rain files → {len(rain)} rows")

# ── 3) MERGE RAIN INTO SONIC BY TIMESTAMP ───────────────────────────────────────

df = pd.merge_asof(
    rain, 
    df,
    on='TIMESTAMP',
    direction='nearest'
)

print(df)

In [ ]:
print(df.columns)

In [ ]:
# ── After merging rain into df via merge_asof ───────────────────────────────────

# 3) Flag rainy periods
df['Flag_Rain'] = (df['Rain'] > 0).astype(int)

# 4) Basic QC filters
#    Flag large T mismatches between sonic and corrected mast temp
df['Temp_Diff']   = df['Temperature_K_2.99'] - df['Average_Temperature_Corr']
df['Flag_Temp']   = (df['Temp_Diff'].abs() > 1).astype(int)
# Windspeed mismatch flag: sonic vs. mast
df['Windspeed_Diff']  = df['WS_ms_D15014_Avg'] - df['Wind_Speed']
df['Flag_Windspeed']  = (df['Windspeed_Diff'].abs() > 1).astype(int)

# 2) Drop any record flagged for rain, temp, or wind‐speed
df = df[
    (df['Flag_Rain']      == 0) &
    (df['Flag_Temp']      == 0) &
    (df['Flag_Windspeed'] == 0)
]

#    Keep only dry, non-spurious records
#df = df[(df['Flag_Rain'] == 0) & (df['Flag_Temp'] == 0)]

# 4) drop NaNs/infs in key fields
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=[
    'uw_flux_corr','vw_flux_corr','wT_Flux',
    'Virtual_Dry_Static_Energy_2','Virtual_Dry_Static_Energy_10',
    'WS_ms_D15008_Avg','WS_ms_D15463_Avg','Dry_Static_Energy_2.99','Average_Temperature_Corr'
])

# 5) compute u*
#df['u_star'] = ((df['uw_flux']**2 + df['vw_flux']**2)**0.5)**0.5


# θ at 2 m and sonic (2.99 m)
df['theta_2']      = df['Virtual_Dry_Static_Energy_2']    
df['theta_sonic']  = df['Virtual_Dry_Static_Energy_10'] 

# 5) Compute flux scales and stability parameters

# friction velocity
df['u_star']     = ((df['uw_flux_corr']**2 + df['vw_flux_corr']**2)**0.5)**0.5

# temperature scale
df['theta_star'] = - df['wT_Flux'] / df['u_star']

# Monin–Obukhov length (using 10 m temp in K)
df['L']          = - df['u_star']**3 / (KAPPA * G / df['Average_Temperature_Corr']  * df['wT_Flux'])

# stability parameter
df['zeta']       = Z_SONIC / df['L']

def classify_stability(zeta):
    if zeta < 0:
        return 'Unstable'
    elif zeta > 0:
        return 'Stable'
    else:
        return 'Near-neutral'

df['stability_class'] = df['zeta'].apply(classify_stability)
print(df['stability_class'].value_counts(normalize=True) * 100)


# 6) Vertical gradients by finite differences

# ∂U/∂z between 2 m and 10 m
df['dU_dz'] = (df['WS_ms_D15463_Avg'] - df['WS_ms_D15008_Avg']) / (10.0 - 2.0)

# ∂θ/∂z between 2 m and 10 m (Temperatures already in K)
df['dT_dz'] = (df['theta_sonic'] - df['theta_2'])     / (10.0  - 2.0)


# 7) Stability functions and final QC

df['phi_m'] = KAPPA * Z_SONIC / df['u_star']     * df['dU_dz']
df['phi_h'] = KAPPA * Z_SONIC / df['theta_star'] * df['dT_dz']

# remove low‐turbulence and extremes
df = df[df['u_star'] > 0.1]
df = df[(df['zeta'] > -1) & (df['zeta'] < 1)]
df = df[(df['phi_m'] > 0) & (df['phi_m'] < 10)]
df = df[(df['phi_h'] > 0) & (df['phi_h'] < 10)]


# 8) Campaign‐wide stability‐function plot

ζ       = np.linspace(-1, 1, 400)
φm_unst = (1 - 15*ζ)**(-1/4)
φh_unst = 0.74*((1 - 9*ζ)**(-1/2))
φm_st   = 1 + 4.7*ζ
φh_st   = 0.74 + 4.7*ζ

#df = df[(df['phi_m'] > 0) & (df['phi_h'] > 0) ]#& (df['u_star'] > 0.1)]


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# φ_m panel
ax1.scatter(df['zeta'], df['phi_m'], s=8, alpha=0.3, label='measured')
ax1.plot(ζ[ζ<0], φm_unst[ζ<0], 'k--', label='BD unstable')
ax1.plot(ζ[ζ>0], φm_st[ζ>0],   'k-',  label='BD stable')
ax1.set(xlabel=r'$\zeta=z/L$', ylabel=r'$\phi_m$', title='Momentum Stability')
ax1.legend(); ax1.grid()

# φ_h panel
ax2.scatter(df['zeta'], df['phi_h'], s=8, alpha=0.3, label='measured')
ax2.plot(ζ[ζ<0], φh_unst[ζ<0], 'k--', label='BD unstable')
ax2.plot(ζ[ζ>0], φh_st[ζ>0],   'k-',  label='BD stable')
ax2.set(xlabel=r'$\zeta=z/L$', ylabel=r'$\phi_h$', title='Heat Stability')
ax2.legend(); ax2.grid()

plt.tight_layout()
plt.show()


In [ ]:
# --- Evaluate deviation from Businger–Dyer curve for phi_m in unstable regime (ζ < -0.1) ---
df_unstable = df[df['zeta'] < 0].copy()
df_unstable['phi_m_BD'] = (1 - 15 * df_unstable['zeta'])**(-1/4)
df_unstable['rel_diff'] = (df_unstable['phi_m'] - df_unstable['phi_m_BD']) / df_unstable['phi_m_BD']

# Median and mode (via binning)
median_rel_diff = df_unstable['rel_diff'].median()
mode_bin = df_unstable['rel_diff'].round(2).mode().iloc[0]

print(f"Median deviation: {median_rel_diff:.2%}")
print(f"Mode deviation: {mode_bin:.2%}")


In [ ]:
# Evaluate deviation from Businger–Dyer curve for phi_m in stable regime (ζ > 0)
df_stable = df[df['zeta'] > 0].copy()
df_stable['phi_m_BD'] = 1 + 4.7 * df_stable['zeta']
df_stable['rel_diff'] = (df_stable['phi_m'] - df_stable['phi_m_BD']) / df_stable['phi_m_BD']

# Median and mode (via binning)
median_rel_diff_stable = df_stable['rel_diff'].median()
mode_bin_stable = df_stable['rel_diff'].round(2).mode().iloc[0]
num_points_stable = len(df_stable)

median_rel_diff_stable, mode_bin_stable, num_points_stable

In [ ]:
df_stable['phi_h_BD']= 0.74 + 4.7*df_stable['zeta']

df_stable['diff'] = (df_stable['phi_h'] - df_stable['phi_h_BD']) / df_stable['phi_h_BD']
print(# Median and mode (via binning)
df_stable['diff'].median(),
df_stable['diff'].round(2).mode().iloc[0])

In [ ]:
from scipy.stats import mode


In [ ]:
# Ensure clear-sky filter is available and remove missing or out-of-bounds values
df_cs = df[
  #  (df['CSI'] > 0.7) &
    (df['zeta'] > -1) & (df['zeta'] < 1) &
    (df['phi_h'] > 0) & (df['phi_h'] < 10)
].copy()

# Define theoretical BD curve for φh
df_cs['phi_h_BD'] = np.where(
    df_cs['zeta'] < 0,
    0.74 * (1 - 9 * df_cs['zeta'])**(-0.5),
    0.74 + 4.7 * df_cs['zeta']
)


# Compute percent deviation
df_cs['phi_h_dev'] = 100 * (df_cs['phi_h'] - df_cs['phi_h_BD']) / df_cs['phi_h_BD']

# Median and mode
median_dev = df_cs['phi_h_dev'].median()
mode_dev = mode(df_cs['phi_h_dev'].round()).mode  # Round to nearest integer for mode

print(f"Median deviation: {median_dev:.2f}%")
print(f"Mode deviation: {mode_dev:.2f}%")
print(f"N points: {len(df_cs)}")

In [ ]:
# Define clear-sky daytime: CSI > 0.7 and SW_down > 50 W/m²
df_clear_sky = df[(df['CSI'] > 0.7)]# & (df['SR15D1Dn_Irr'] > 50)]  # Replace 'SW↓' with actual column name
df_cs_unstable = df_clear_sky[df_clear_sky['zeta'] < -0.1].copy()
df_cs_unstable['phi_m_BD'] = (1 - 15 * df_cs_unstable['zeta'])**(-1/4)
df_cs_unstable['rel_diff'] = (df_cs_unstable['phi_m'] - df_cs_unstable['phi_m_BD']) / df_cs_unstable['phi_m_BD']
median_cs = df_cs_unstable['rel_diff'].median()
mode_cs = df_cs_unstable['rel_diff'].round(2).mode().iloc[0]
within_15_cs = (df_cs_unstable['rel_diff'].abs() <= 0.15).mean() * 100
print(f"Median deviation: {median_cs:.2%}")
print(f"Mode deviation: {mode_cs:.2%}")
print(within_15_cs)

In [ ]:

# Create figure and subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), sharey=True)

# Common plot parameters
scatter_kwargs = dict(s=20, alpha=0.5, edgecolors='none')
line_kwargs_stable = dict(color='black', linewidth=2.5, label='BD stable')
line_kwargs_unstable = dict(color='black', linestyle='--', linewidth=2.5, label='BD unstable')

# --- φ_m subplot ---
ax1.scatter(df['zeta'], df['phi_m'], **scatter_kwargs, label='Measured')
ax1.plot(ζ[ζ < 0], φm_unst[ζ < 0], **line_kwargs_unstable)
ax1.plot(ζ[ζ > 0], φm_st[ζ > 0], **line_kwargs_stable)
ax1.set_xlabel(r'$\zeta = z/L$', fontsize=20)
ax1.set_ylabel(r'$\phi_m$', fontsize=20)
ax1.set_title('Momentum Stability Function', fontsize=20)
ax1.tick_params(labelsize=18)
ax1.legend(loc='upper left', fontsize=18)
ax1.grid(True)

# --- φ_h subplot ---
ax2.scatter(df['zeta'], df['phi_h'], **scatter_kwargs, label='Measured')
ax2.plot(ζ[ζ < 0], φh_unst[ζ < 0], **line_kwargs_unstable)
ax2.plot(ζ[ζ > 0], φh_st[ζ > 0], **line_kwargs_stable)
ax2.set_xlabel(r'$\zeta = z/L$', fontsize=20)
ax2.set_ylabel(r'$\phi_h$', fontsize=20)
ax2.set_title('Heat Stability Function', fontsize=20)
ax2.tick_params(labelsize=18)
ax2.legend(loc='upper left', fontsize=18)
ax2.grid(True)

# Final layout
#fig.suptitle('Stability Functions Compared to Businger–Dyer Curves', fontsize=22)
plt.tight_layout(rect=[0, 0, 1, 0.95])
output_path = os.path.join(DATA_ROOT, 'stability_functions.jpg')
plt.savefig(output_path, dpi=300)  # save at high resolution for report
plt.show()
